In [334]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [794]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [760]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [761]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [762]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [763]:
print(train['Sex'].value_counts())
print(test['Sex'].value_counts(), '\n')
train['Sex_Num'] = train['Sex'].astype('category').cat.codes
test['Sex_Num'] = test['Sex'].astype('category').cat.codes
print(train['Sex_Num'].unique())
print(test['Sex_Num'].unique())
train.drop(['Sex'], axis=1, inplace=True)
test.drop(['Sex'], axis=1, inplace=True)

male      577
female    314
Name: Sex, dtype: int64
male      266
female    152
Name: Sex, dtype: int64 

[1 0]
[1 0]


In [764]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Major', 'Col', 'Don', 'Countess', 'Jonkheer', 'Capt', 'Sir', 'Mlle', 'Mme', 'Lady', 'Ms'], 'Other')
test['Title'] = test['Title'].replace(['Col', 'Rev', 'Dr', 'Ms', 'Dona'], 'Other')

train['Title_Num'] = train['Title'].astype('category').cat.codes
test['Title_Num'] = test['Title'].astype('category').cat.codes


print(train['Title'].value_counts(), '\n',train['Title_Num'].value_counts(), '\n')
print(test['Title'].value_counts(), '\n', test['Title_Num'].value_counts())

train.drop(['Name', 'Title'], axis=1, inplace=True)
test.drop(['Name', 'Title'], axis=1, inplace=True)

Mr        517
Miss      182
Mrs       125
Master     40
Other      27
Name: Title, dtype: int64 
 2    517
1    182
3    125
0     40
4     27
Name: Title_Num, dtype: int64 

Mr        240
Miss       78
Mrs        72
Master     21
Other       7
Name: Title, dtype: int64 
 2    240
1     78
3     72
0     21
4      7
Name: Title_Num, dtype: int64


In [765]:
train.drop(['Ticket'], axis=1, inplace=True)
test.drop(['Ticket'], axis=1, inplace=True)

In [766]:
train.drop(['Cabin'], axis=1, inplace=True)
test.drop(['Cabin'], axis=1, inplace=True)

In [767]:
test['Fare'] = test['Fare'].fillna(test[test['Pclass']==3]['Fare'].mean())

In [768]:
train['Embarked'] = train['Embarked'].fillna('S')
train['Embarked_Num'] = train['Embarked'].astype('category').cat.codes
test['Embarked_Num'] = test['Embarked'].astype('category').cat.codes

print(train['Embarked_Num'].unique(), '\n', train['Embarked_Num'].value_counts(), '\n', train['Embarked'].value_counts())
print(test['Embarked_Num'].unique(), '\n', test['Embarked_Num'].value_counts(), '\n', test['Embarked'].value_counts())

train.drop(['Embarked'], axis=1, inplace=True)
test.drop(['Embarked'], axis=1, inplace=True)

[2 0 1] 
 2    646
0    168
1     77
Name: Embarked_Num, dtype: int64 
 S    646
C    168
Q     77
Name: Embarked, dtype: int64
[1 2 0] 
 2    270
0    102
1     46
Name: Embarked_Num, dtype: int64 
 S    270
C    102
Q     46
Name: Embarked, dtype: int64


In [769]:
train['Family'] = train['SibSp'] + train['Parch'] + 1
test['Family'] = test['SibSp'] + test['Parch'] + 1
train.drop(['SibSp', 'Parch'], axis=1, inplace=True)
test.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [770]:
train.drop(['PassengerId'], axis=1, inplace=True)

In [771]:
train['Age'] = train['Age'].fillna(train.groupby(['Title_Num'])['Age'].transform('median'))
test['Age'] = test['Age'].fillna(train.groupby('Title_Num')['Age'].transform('median'))

In [772]:
train.loc[ train['Age'] <= 10, 'Age_Num'] = 0
train.loc[(train['Age'] > 10) & (train['Age'] <= 16), 'Age_Num'] = 1
train.loc[(train['Age'] > 16) & (train['Age'] <= 20), 'Age_Num'] = 2
train.loc[(train['Age'] > 20) & (train['Age'] <= 26), 'Age_Num'] = 3
train.loc[(train['Age'] > 26) & (train['Age'] <= 30), 'Age_Num'] = 4
train.loc[(train['Age'] > 30) & (train['Age'] <= 36), 'Age_Num'] = 5
train.loc[(train['Age'] > 36) & (train['Age'] <= 40), 'Age_Num'] = 6
train.loc[(train['Age'] > 40) & (train['Age'] <= 46), 'Age_Num'] = 7
train.loc[(train['Age'] > 46) & (train['Age'] <= 50), 'Age_Num'] = 8
train.loc[(train['Age'] > 50) & (train['Age'] <= 60), 'Age_Num'] = 9
train.loc[ train['Age'] > 60, 'Age_Num'] = 10
train.drop(['Age'], axis=1, inplace=True)

test.loc[ test['Age'] <= 10, 'Age_Num'] = 0
test.loc[(test['Age'] > 10) & (test['Age'] <= 16), 'Age_Num'] = 1
test.loc[(test['Age'] > 16) & (test['Age'] <= 20), 'Age_Num'] = 2
test.loc[(test['Age'] > 20) & (test['Age'] <= 26), 'Age_Num'] = 3
test.loc[(test['Age'] > 26) & (test['Age'] <= 30), 'Age_Num'] = 4
test.loc[(test['Age'] > 30) & (test['Age'] <= 36), 'Age_Num'] = 5
test.loc[(test['Age'] > 36) & (test['Age'] <= 40), 'Age_Num'] = 6
test.loc[(test['Age'] > 40) & (test['Age'] <= 46), 'Age_Num'] = 7
test.loc[(test['Age'] > 46) & (test['Age'] <= 50), 'Age_Num'] = 8
test.loc[(test['Age'] > 50) & (test['Age'] <= 60), 'Age_Num'] = 9
test.loc[ test['Age'] > 60, 'Age_Num'] = 10
test.drop(['Age'], axis=1, inplace=True)

In [773]:
train['Fare'] = pd.qcut(train['Fare'], 5)
test['Fare'] = pd.qcut(test['Fare'], 5)

train['Fare_Num'] = train['Fare'].astype('category').cat.codes
test['Fare_Num'] = train['Fare'].astype('category').cat.codes

print(train['Fare'].value_counts(), '\n', train['Fare_Num'].unique())
print(test['Fare'].value_counts(), '\n', test['Fare_Num'].unique())

train.drop('Fare', axis=1, inplace=True)
test.drop('Fare', axis=1, inplace=True)

(7.854, 10.5]        184
(21.679, 39.688]     180
(-0.001, 7.854]      179
(39.688, 512.329]    176
(10.5, 21.679]       172
Name: Fare, dtype: int64 
 [0 4 1 2 3]
(-0.001, 7.796]     85
(11.342, 21.196]    84
(46.34, 512.329]    84
(21.196, 46.34]     83
(7.796, 11.342]     82
Name: Fare, dtype: int64 
 [0 4 1 2 3]


In [776]:
test

,PassengerId,Pclass,Sex_Num,Title_Num,Embarked_Num,Family,Age_Num,Fare_Num
0,892,3,1,2,1,1,5.0,0
1,893,3,0,3,2,2,8.0,4
2,894,2,1,2,1,1,10.0,1
3,895,3,1,2,2,1,4.0,4
4,896,3,0,3,2,3,3.0,1
...,...,...,...,...,...,...,...,...
413,1305,3,1,2,2,1,4.0,0
414,1306,1,0,4,0,1,6.0,1
415,1307,3,1,2,2,1,6.0,1
416,1308,3,1,2,2,1,5.0,3


In [ ]:
# survival	Survival	0 = No, 1 = Yes
# pclass Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
# sex Sex	
# Age Age in years	
# sibsp # of siblings / spouses aboard the Titanic	
# parch # of parents / children aboard the Titanic	
# ticket Ticket number	
# fare Passenger fare	
# cabin Cabin number	
# embarked Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
###

In [784]:
X_train = train.drop(['Survived'], axis=1).copy()
y_train = train['Survived']
X_test = test.drop(['PassengerId'], axis=1).copy()

X_train.shape, y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [786]:
Feature = ['Pclass', 'Sex_Num', 'Title_Num', 'Embarked_Num', 'Family', 'Age_Num', 'Fare_Num']
Label = ['Survived']

In [787]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg_preds = logreg.predict(X_test)

acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

79.35

In [797]:
svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)

acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

83.39

In [804]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

87.77

In [805]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
gaus_pred = gaussian.predict(X_test)

acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
acc_gaussian

79.24

In [811]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
perceptron_pred = perceptron.predict(X_test)

acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
acc_perceptron

75.87

In [812]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
linear_svc_pred = linear_svc.predict(X_test)

acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)
print(acc_linear_svc)

79.01


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [825]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_test)

acc_sgd = round(sgd.score(X_train, y_train) * 100, 2)
acc_sgd

77.55

In [827]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
decision_pred = decision_tree.predict(X_test)

acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

90.68

In [828]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
randomforest_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

90.68

In [829]:
submission_1 = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": decision_pred
    })

In [830]:
submission_2 = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": randomforest_pred
    })

In [856]:
submission_1.to_csv("./submission_1.csv",index=False)
submission_2.to_csv("./submission_2.csv",index=False)

In [858]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

data = train[Feature]
target = train[Label]

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy').mean()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y wa

0.8361173533083643

In [859]:
X_train = train[Feature]
X_test = test[Feature]
y_train = train[Label]

clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)
clf.fit(X_train, y_train)
submission['Survived'] = clf.predict(X_test)
submission.to_csv('submission_3.csv',index=False)

<ipython-input-859-55042a3030c1>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
